In [2]:
import wrds
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [47]:
def pull_option_price(wrds_username, year=2012): 
	db = wrds.Connection(wrds_username=wrds_username)

	sql_query = f"""
					SELECT 
						*
					FROM
						optionm_all.opprcd{year} AS a
					WHERE
						a.secid=108105;
				"""
		
	optm_df = db.raw_sql(sql_query, date_cols = ["date"])

	db.close()

	return optm_df

def pull_all_option_price(wrds_username, last_year=2012): 
	db = wrds.Connection(wrds_username=wrds_username)

	optm_df = []

	for year in range(1996,last_year+1):
		sql_query = f"""
						SELECT 
							a.date, a.exdate, a.cp_flag, a.strike_price
						FROM
							optionm_all.opprcd{year} AS a
						WHERE
							a.secid=108105;
					"""
		
		optm = db.raw_sql(sql_query, date_cols = ["date"])

		optm_df.append(optm)
		
	db.close()

	optm_df = pd.concat(optm_df)
	
	return optm_df


def pull_all_req_data(wrds_username, start_year=1996, end_year=2012): 
	db = wrds.Connection(wrds_username=wrds_username)

	optm_df = []

	for year in range(start_year, end_year+1):
		sql_query = f"""
						SELECT 
							a.date, a.exdate, a.cp_flag, a.strike_price, a.best_bid, a.best_offer,
							b.open, b.close, a.impl_volatility, c.tb_m3
						FROM
							optionm_all.opprcd{year} AS a
						JOIN 
							optionm_all.secprd{year} AS b ON a.date = b.date AND a.secid = b.secid
						JOIN 
							frb_all.rates_daily AS c ON c.date = a.date 
						WHERE
							a.secid=108105 AND a.date <= '2012-01-31';
					"""
		
		optm = db.raw_sql(sql_query, date_cols = ["date"])

		optm_df.append(optm)
		
	db.close()

	optm_df = pd.concat(optm_df)
	
	return optm_df

In [125]:
optm_df = pull_all_req_data("hholt")

Loading library list...
Done


In [126]:
print(optm_df.shape)
print(optm_df['cp_flag'].value_counts().to_dict())

(3410580, 10)
{'P': 1706360, 'C': 1704220}


In [127]:
optm_df['date'] = pd.to_datetime(optm_df['date'])
optm_df['option_price'] = (optm_df['best_bid'] + optm_df['best_offer'])/2
optm_df['index_price'] = (optm_df['open'] + optm_df['close'])/2
optm_df['strike_price'] = optm_df['strike_price']/1000

### Level 1 filters

#### Identical Filter:
The OptionMetrics data set contain duplicate observations,
defined as two or more quotes with identical option type, strike, expiration
date, and price. In each such case, we eliminate all but one of the quotes.

In [128]:
# Assuming 'df' is your DataFrame

def delete_identical_filter(df):
    columns_to_check = ['cp_flag', 'strike_price','date', 'exdate', 'option_price']

    # Drop duplicates based on specified columns
    df = df.drop_duplicates(subset=columns_to_check, keep='first')

    return df

In [130]:
optm_l1_df = delete_identical_filter(optm_df)
print(optm_l1_df.shape)

test_identical1 = optm_df[optm_df.duplicated(columns_to_check,keep=False)]
print(test_identical1.shape[0])

(3410579, 12)
2


#### Identical Except Price Filter:
There are a few sets of quotes with identical
terms (type, strike, and maturity) but different prices. When this occurs, we
keep the quote whose T-bill-based implied volatility is closest to that of its
moneyness neighbors, and delete the others.

NEXT STEPS - determine moneyness neighbors >> only 18 duplicates so 9 deleted versus 11 in paper

In [131]:
columns_to_check = ['cp_flag', 'strike_price','date', 'exdate']

identical_df = optm_l1_df[optm_l1_df.duplicated(columns_to_check,keep=False)].sort_values(columns_to_check)

identical_df.shape

(18, 12)

In [132]:
identical_df = optm_l1_df[optm_l1_df.duplicated(columns_to_check,keep='first')].sort_values(columns_to_check)
identical_df.shape

(9, 12)

In [38]:
conn = wrds.Connection(wrds_username="hholt")
conn.list_tables(library="optionm_all")

Loading library list...
Done


['distrd',
 'distrprojd1996',
 'distrprojd1997',
 'distrprojd1998',
 'distrprojd1999',
 'distrprojd2000',
 'distrprojd2001',
 'distrprojd2002',
 'distrprojd2003',
 'distrprojd2004',
 'distrprojd2005',
 'distrprojd2006',
 'distrprojd2007',
 'distrprojd2008',
 'distrprojd2009',
 'distrprojd2010',
 'distrprojd2011',
 'distrprojd2012',
 'distrprojd2013',
 'distrprojd2014',
 'distrprojd2015',
 'distrprojd2016',
 'distrprojd2017',
 'distrprojd2018',
 'distrprojd2019',
 'distrprojd2020',
 'distrprojd2021',
 'distrprojd2022',
 'distrprojd2023',
 'exchgd',
 'fwdprd1996',
 'fwdprd1997',
 'fwdprd1998',
 'fwdprd1999',
 'fwdprd2000',
 'fwdprd2001',
 'fwdprd2002',
 'fwdprd2003',
 'fwdprd2004',
 'fwdprd2005',
 'fwdprd2006',
 'fwdprd2007',
 'fwdprd2008',
 'fwdprd2009',
 'fwdprd2010',
 'fwdprd2011',
 'fwdprd2012',
 'fwdprd2013',
 'fwdprd2014',
 'fwdprd2015',
 'fwdprd2016',
 'fwdprd2017',
 'fwdprd2018',
 'fwdprd2019',
 'fwdprd2020',
 'fwdprd2021',
 'fwdprd2022',
 'fwdprd2023',
 'hvold1996',
 'hvold1997'

In [39]:
conn.get_table(library="optionm_all", table="opprcd1996")

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix
0,5005.0,1996-01-04,09C04.71,0,1996-05-18,None,C,5000.0,3.500,3.8750,0.0,10.0,0.499353,0.977071,0.021003,0.282112,-0.450118,10224753.0,1.0,0.0,100.0,0,None,None,09C04,71
1,5005.0,1996-01-04,099AF.44,0,1996-08-17,None,P,7500.0,0.750,0.9375,0.0,10.0,0.565265,-0.282751,0.091322,2.264206,-0.893695,10071876.0,1.0,0.0,100.0,0,None,None,099AF,44
2,5005.0,1996-01-04,09B86.5D,0,1996-02-17,None,C,10000.0,0.250,0.3750,0.0,423.0,0.652115,0.290704,0.178798,1.006999,-2.910794,10192477.0,1.0,0.0,100.0,0,None,None,09B86,5D
3,5005.0,1996-01-04,09D85.65,0,1996-02-17,None,C,5000.0,3.375,3.7500,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10323301.0,1.0,0.0,100.0,0,None,None,09D85,65
4,5005.0,1996-01-04,09FB8.95,0,1996-01-20,1996-01-04,C,10000.0,0.000,0.1250,40.0,200.0,0.618655,0.123897,0.190533,0.355180,-2.730374,10467477.0,1.0,0.0,100.0,0,None,None,09FB8,95


In [41]:
stdop = conn.get_table(library="optionm_all", table="stdopd1996")

In [42]:
stdop = stdop.loc[stdop['secid']==108105]

In [ ]:
df = pull_Option_info("hholt")

In [ ]:
opinfd.loc[opinfd['secid']==108105]

In [ ]:
df.head()

In [ ]:
dup_cols = ['cp_flag','strike_price']

duplicates = df[df.duplicated(subset=dup_cols, keep=False)]


In [ ]:
duplicates.head()